# TED web scraper

The purpose of this notebook is to create a web scraper that collects TED talks:
- slugs (url part) by topic
- info (talk and speaker details, including transcripts) by slugs





In [2]:
# Importing libraries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import json
import time
import os

---
### 1. Downloding slug(s) by topic
slug = url element to a talk

There are options available:<br>
A. **Version 1** - using api request<br> 
B. **Version 2** - parsing html using beautiful soup -> might not work everytime<br>

In [8]:
# User Inputs

topic='machine learning' # Select a topic from the list available here: https://www.ted.com/topics

filename=f"slug/TED_Talk_{topic}_URLs.txt"


In [4]:
# Version 1 - using api request

def api_scraping(topic, filename):
    urls = []
    page_number=0
    total_pages=0
    f = open(filename, 'w')
    
    while True:
        # The API endpoint to request
        endpoint = "https://zenith-prod-alt.ted.com/api/search"

        # The headers for the request
        headers = {
        "Accept": "*/*",
        "Content-Type": "application/json"
      }

      # The data for the request
        data = [{
              "indexName": "coyote_models_acme_videos_alias_21e1372f285984be956cd03b7ad3406e",
              "params": {
                  "attributeForDistinct": "objectID",
                  "distinct": 1,
                  "facetFilters": [
                      [
                          "tags:"+topic
                      ]
                  ],
                  "facets": [
                      "subtitle_languages",
                      "tags"
                  ],
                  "highlightPostTag": "__/ais-highlight__",
                  "highlightPreTag": "__ais-highlight__",
                  "hitsPerPage": 24,
                  "maxValuesPerFacet": 500,
                  "page": page_number,
                  "query": "",
                  "tagFilters": ""
              }
          },
          {
              "indexName": "coyote_models_acme_videos_alias_21e1372f285984be956cd03b7ad3406e",
              "params": {
                  "analytics": False,
                  "attributeForDistinct": "objectID",
                  "clickAnalytics": False,
                  "distinct": 1,
                  "facets": "tags",
                  "highlightPostTag": "__/ais-highlight__",
                  "highlightPreTag": "__ais-highlight__",
                  "hitsPerPage": 0,
                  "maxValuesPerFacet": 500,
                  "page": page_number,
                  "query": ""
              }
          }]



        # Send the POST request and get the response
        response = requests.post(endpoint, headers=headers, data=json.dumps(data))
        if page_number==0:
            total_pages=response.json()['results'][0]['nbPages']

        for slug in response.json()['results'][0]['hits']:
            urls.append(slug['slug'])
        print(f"{page_number} / {total_pages}")
        page_number+=1
        if page_number>total_pages:
            break
    f.write('\n'.join(urls))
    f.close()

    print(f"Done.{len(urls)} URLs for topic {topic} have been saved in {f}.")
    return

In [9]:
# Version 2 - using beautiful soup parsing

def html_scrapping(topic, filename):
    urls = []
    page_number=1
    f = open(filename, 'w')
    
    while True:
        res  =  requests.get("https://www.ted.com/talks?page="+str(page_number)+"&sort=newest&topics[]="+topic)

        soup = BeautifulSoup(res.text)
        e=soup.find('div', id='browse-results')

        string_end="Sorry. We couldn't find a talk quite like that." 

        if e is None:
            break
        elif e.find('div', class_='h3 m2') is not None:
            if e.find('div', class_='h3 m2').text==string_end:
                break
        else:
            for elem in e.find_all("h4", class_="h9 m5 f-w:700"):
                urls.append(elem.find('a')['href'])
            page_number+=1


    f.write('\n'.join(urls))
    f.close()



In [9]:
api_scraping(topic,filename)

0 / 4
1 / 4
2 / 4
3 / 4
4 / 4
Done.79 URLs for topic machine learning have been saved in <_io.TextIOWrapper name='slug/TED_Talk_machine learning_URLs.txt' mode='w' encoding='cp1252'>.


---
### 2. Downloading content for each talk

In [6]:
#functions

# function to get id

def getBuildID():
    response=requests.get("https://ted.com")
    buildID=str(response.content).split("buildId\":\"")[1].split("\"")[0]
    return buildID

# function to build url

def buildDataURL(slug):
    daily_id=getBuildID()
    base=f"https://www.ted.com/_next/data/{daily_id}/talks/"
    mid=".json?slug="
    url=base+slug+mid+slug
    return url

# function to get slug data

def getSlugData(url):
    response=requests.get(url)
    try:
        return response.json()
    except ValueError:
        print(url)
        print(response)
        issues[url]=response

# function to export json data for a list of slugs

def extract_json_by_slug(path_to_json,slugs, retry_limit=3):
    count=0
    max_count=10

    data_list=[]
    slugs_retry={}

    for i, slug in enumerate(slugs):
        count+=1
        slugURL = buildDataURL(slug)
        data_json = getSlugData(slugURL)

        if data_json is None: #checking if the respose is null
            if slug in slugs_retry:
                slugs_retry[slug] += 1
            else:
                slugs_retry[slug] = 1
        else:
            json_object = json.dumps(data_json)
            file_slug=slug if len(slug)<100 else slug[:101]
            
            with open(f"{path_to_json}{file_slug}.json", "w") as outfile:
                outfile.write(json_object)

        # adding throttling/wait time to stay within rate rate limiting parameters     
        if count>max_count:
            time.sleep(10)
            count=0
        else:
            time.sleep(3)
        
        # Print progress for every 100 slugs processed
        if (i+1) % 100 == 0:
            print(f"Processed {i+1} out of {len(slugs)} slugs.")
    
    # Retry slugs that failed but haven't reached the retry limit
    slugs_to_retry = [slug for slug, retries in slugs_retry.items() if retries < retry_limit]
    
    print(f"There are {len(slugs_to_retry)} slugs that still return None after retries.")
    
    if len(slugs_to_retry)!=0:
        extract_json_by_slug(slugs_to_retry, retry_limit)
        
    if len(issues) > 0:
        print(issues)
    return slugs_to_retry,issues


# Data quality check: function to check for null jsons and missing transcripts
missing=[]
nulls=[]

def check_null_missing_transcript(path_to_json):
    count_missing=0
    count_null=0
    
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    for jfile in json_files:
        with open(os.path.join(path_to_json, jfile)) as json_file:
            json_text = json.load(json_file)
            if json_text is None:
                count_null+=1
                nulls.append(jfile)
            elif json_text["pageProps"]["transcriptData"]["translation"] is None:
                count_missing+=1
                missing.append(jfile)
    print("Summary:")        
    print(f"There are {count_null} null jsons files")
    print(f"There are {count_missing} jsons files with missing transcripts")



In [3]:
# define which slugs to be loaded and where to save json files
path_to_json = 'jsons/'
path_to_slug = 'slug/'
slug_files = [pos_slug for pos_slug in os.listdir(path_to_slug) if pos_slug.endswith('.txt')]

slugs_combined=[]

for sfile in slug_files:
    print(sfile)
    with open(os.path.join(path_to_slug, sfile)) as slug_file:
        loaded=slug_file.read().splitlines()
        print(f"{len(loaded)} slugs found")
        slugs_combined.append(loaded)

# combining all slugs and removing duplicates        
slugs = list(set([item for sublist in slugs_combined for item in sublist]))


TED_Talk_3D printing_URLs.txt
14 slugs found
TED_Talk_Activism_URLs.txt
391 slugs found
TED_Talk_Addiction_URLs.txt
22 slugs found
TED_Talk_Africa_URLs.txt
216 slugs found
TED_Talk_Aging_URLs.txt
66 slugs found
TED_Talk_Agriculture_URLs.txt
87 slugs found
TED_Talk_AIDS_URLs.txt
25 slugs found
TED_Talk_Ai_URLs.txt
144 slugs found
TED_Talk_Algorithm_URLs.txt
72 slugs found
TED_Talk_Aliens_URLs.txt
24 slugs found
TED_Talk_Alzheimer's_URLs.txt
17 slugs found
TED_Talk_Ancient world_URLs.txt
109 slugs found
TED_Talk_Animals_URLs.txt
295 slugs found
TED_Talk_Animation_URLs.txt
935 slugs found
TED_Talk_Antarctica_URLs.txt
15 slugs found
TED_Talk_Anthropocene_URLs.txt
37 slugs found
TED_Talk_Anthropology_URLs.txt
66 slugs found
TED_Talk_Archaeology_URLs.txt
29 slugs found
TED_Talk_Architecture_URLs.txt
139 slugs found
TED_Talk_Art_URLs.txt
475 slugs found
TED_Talk_Asia_URLs.txt
43 slugs found
TED_Talk_Asteroid_URLs.txt
8 slugs found
TED_Talk_Astrobiology_URLs.txt
18 slugs found
TED_Talk_Astrono

In [4]:
print(len(slugs))

6099


In [21]:
issues={}
extract_json_by_slug(path_to_json,slugs)

Processed 100 out of 1364 slugs.
Processed 200 out of 1364 slugs.
Processed 300 out of 1364 slugs.
Processed 400 out of 1364 slugs.
Processed 500 out of 1364 slugs.
<Response [429]>
Processed 600 out of 1364 slugs.
Processed 700 out of 1364 slugs.
Processed 800 out of 1364 slugs.
Processed 900 out of 1364 slugs.
Processed 1000 out of 1364 slugs.
Processed 1100 out of 1364 slugs.
Processed 1200 out of 1364 slugs.
Processed 1300 out of 1364 slugs.
There are 1 slugs that still return None after retries.
There are 0 slugs that still return None after retries.


['balsher_singh_sidhu_are_we_running_out_of_clean_water']

---
### 3. Checking for missing transcripts & null json files

In [7]:
check_null_missing_transcript(path_to_json)

Summary:
There are 0 null jsons files
There are 676 jsons files with missing transcripts


In [8]:
pct=len(missing)*100/len(slugs)

print(f"{round(pct,1)}% of talks do not have a transcript")


11.1% of talks do not have a transcript


> Next: to parse the json files and create the dataframe